## 下载最新schema文件并提取出最新form的完整结构

In [ ]:
! curl -o schema.ts "https://raw.githubusercontent.com/IKKEM-Lin/annotation-demo/refs/heads/main/src/schema.ts"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 61904  100 61904    0     0   100k      0 --:--:-- --:--:-- --:--:--  100k


In [ ]:
import json
import re
with open('schema.ts', 'r', encoding='utf-8') as f:
    schema = f.read()
fix_str = re.sub(r"export\s*const\s*schema\s*=\s*", "", schema.strip())
fix_str = re.sub(r";$", "", fix_str)
schema = json.loads(fix_str).get("schema")

def generate_form_structure(schema):
    if schema.get("type") == "void" and not schema.get("properties"):
      return None
    elif schema.get("type") in ["object", "void"]:
        form_structure = {}
        properties = schema.get("properties", {})
        for key, value in properties.items():
            child_strc = generate_form_structure(value)
            if value.get("type") == "void" and type(child_strc) == dict:
              for child_key, child_value in child_strc.items():
                if child_value:
                  form_structure[child_key] = child_value
            else:
              form_structure[key] = child_strc
        return form_structure

    elif schema.get("type") == "array":
        items = schema.get("items", {})
        return [generate_form_structure(items)]
    else:
        return "unknown"

form_structure = generate_form_structure(schema)

# print(json.dumps(form_structure, indent=4, ensure_ascii=False))

## 根据提供的用户名密码，自动拉取该用户的所有任务的最新标注结果

In [ ]:
import requests

# api返回结果按照form结构填充
def data_to_form(data, structure, data_is_struc = False):
  if isinstance(data, list):
    return [data_to_form(item, structure[0], data_is_struc) for item in data]
  elif isinstance(data, dict):
    form_data = {}
    for key, value in structure.items():
      # if isinstance(value, list):
      #   print(key)
      skip_struc_fill = isinstance(value, list) and data_is_struc
      new_data_is_struc = data_is_struc or (key not in data)
      form_data[key] = data_to_form(not skip_struc_fill and data.get(key, value) or [], value, new_data_is_struc)
    return form_data
  else:
    return data

def main(username, password, domin):
  login_url = f"{domin}/api/login"
  task_url = f"{domin}/api/upload-json"

  login_res = requests.post(login_url, headers={}, json={
    "username": username,
    "password": password
  })
  authorization = login_res.json()['token']

  task_res = requests.get(task_url, headers={ "authorization": authorization, "content-type": "application/json", })
  tasks = task_res.json()
  # print(tasks)
  pending_tasks = []
  for doi, val in tasks.items():
    if val.get("latestFile"):
      pending_tasks.append({"doi": doi, "file": val.get("latestFile")})

  result = []
  for task in pending_tasks:
    article_name = f'{task.get("doi")}.pdf'
    file_url = f'{domin}/api/upload-json/{article_name}/{task.get("file")}'
    res = requests.get(file_url, headers={ "authorization": authorization, "content-type": "application/json", })
    result.append({"doi": task["doi"], "data": data_to_form(res.json()[0]["value"], form_structure)})
  return result

In [ ]:
domin = "http://192.168.1.219:9557"

data = {
    "username": "",
    "password": ""
}

print(json.dumps(main(data['username'], data['password'], domin), indent=4, ensure_ascii=False))

[
    {
        "doi": "10.1002_anie.201608165",
        "data": {
            "reactionType": "thermal catalysis",
            "reactants": {
                "reactantList": [
                    {
                        "reactantName": "unknown",
                        "conversion": "unknown"
                    }
                ],
                "reactantMixtureRatio": "tes"
            },
            "catalyst": {
                "catalystName": "unknown",
                "catalystMixtureRatio": "unknown",
                "promoters": [],
                "catalystBETSurfaceArea": "unknown",
                "turnoverFrequency": "unknown",
                "catalystLoading": "unknown",
                "catalystSupport": "unknown",
                "catalystShape": "unknown",
                "catalystPoreSize": "unknown",
                "catalystPoreVolume": "unknown",
                "catPreparationMethod": "unknown",
                "catAcidityAndBasicity": "unknown",
           